In [ ]:
test_file = "10_cols_1_mb"
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("LargeRowBenchmark").getOrCreate()

In [ ]:
df = spark.read.parquet(f"data/openai/parquet/{test_file}.parquet")
df.createOrReplaceTempView("T")
df.printSchema()
df.show(3)

In [ ]:
# Q1. get the length of structure's long string
spark.sql("SELECT length(json_0['text']) FROM T").collect()

In [ ]:
# Q2. find word in structure's long string
spark.sql("SELECT id, json_0 FROM T WHERE contains(json_0['text'], 'House')").collect()

In [ ]:
# Q3. lowercase the structure's long string
spark.sql("SELECT lower(json_0['text']) FROM T").collect()

In [ ]:
# Q4. UDF: count word frequency in structure's long string
def word_count(text):
    word_count = {}
    for word in text.split():
        if word in word_count:
            word_count[word] += 1
        else:
            word_count[word] = 1
    return word_count


spark.udf.register("word_count", word_count)
spark.sql("SELECT word_count(json_0['text']) FROM T").collect()

In [7]:
spark.stop()